In [51]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

#to-do list
#1. Record the amount of time a classifier takes: start(timestamp) - end(timestamp), put it in .pkl.gz

#Observations:
# 1. RF gives highest accuracy, but takes a lot of time to train: 25 minutes and 15 minutes
# 2. Neural Network is weakest
# 3. NB gives satisfactory results within a minute.

trainDF = pd.concat([pd.read_pickle('../../data/2015/MasterData_2015.pkl.gz'),
                    pd.read_pickle('../../data/2014/MasterData_2014.pkl.gz'),
                    pd.read_pickle('../../data/2013/MasterData_2013.pkl.gz'),
                    pd.read_pickle('../../data/2012/MasterData_2012.pkl.gz')])
                                   
trainDF = trainDF[trainDF.TEXT.notna() & trainDF.NTEE.notna()]
trainDF['text'] = trainDF['TEXT'].astype(str)
trainDF['label'] = trainDF['NTEE'].astype(str)
trainDF['category'] = (trainDF['NTEE'].apply(ord)-64).astype('float32')

#To remove error Unsupported feed type from tf
#print(type(trainDF['category'].values[0]))

In [52]:
trainDF = trainDF.drop(['EIN', 'NTEE', 'IRS_URL', 'TEXT','TEXTTYPE', 'YEAR', 'label'], axis=1)
train_df, test_df = np.split(trainDF, [int(.7*len(trainDF))])

#tf.logging.set_verbosity(tf.logging.ERROR)


In [53]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df['category'], num_epochs=None, shuffle=True)

# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df['category'], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    test_df, test_df['category'], shuffle=False)

In [65]:
#Token based text embedding trained on English Google News 200B corpus.
embedded_text_feature_column = hub.text_embedding_column(
    key="text", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")


#DNN Regressor - for multiclass classifier, Use DNNClassifier for binaru classification.
estimator = tf.estimator.DNNRegressor(
    hidden_units=[500, 100],
    feature_columns=[embedded_text_feature_column],
#    optimizer=lambda: tf.AdamOptimizer(
#        learning_rate=tf.exponential_decay(
#            learning_rate=0.1,
#            global_step=tf.get_global_step(),
#            decay_steps=10000,
#            decay_rate=0.96))

   optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001))


# Training for 1,000 steps means 128,000 training examples with the default
# batch size. This is roughly equivalent to 5 epochs since the training dataset
# contains 25,000 examples. 365390
    
estimator.train(input_fn=train_input_fn, steps=100000);

train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

train_pred_result = estimator.predict(input_fn=predict_train_input_fn)
test_pred_result = estimator.predict(input_fn=predict_test_input_fn)

print(train_eval_result)
print(test_eval_result)
    
print(train_pred_result)
print(test_pred_result)

{'average_loss': 24.739607, 'label/mean': 10.442828, 'loss': 3166.2363, 'prediction/mean': 10.821727, 'global_step': 100000}
{'average_loss': 29.57418, 'label/mean': 10.4608135, 'loss': 3783.683, 'prediction/mean': 10.893915, 'global_step': 100000}
<generator object Estimator.predict at 0x1581c5888>
<generator object Estimator.predict at 0x1320a9bf8>
